In [1]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp
from pennylane.optimize import AdamOptimizer

# General imports
import os
import json
import numpy as np
from datetime import datetime

from qiskit.quantum_info import SparsePauliOp

#custom module
from susy_qm import calculate_wz_hamiltonian


In [3]:
# Parameters
N = 2
a = 1.0
c = 0
potential = "linear"#'quadratic'
boundary_condition = 'dirichlet'#'periodic'

In [ ]:
cut_offs_list = [4]
shots = None

starttime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

if shots != None:
    folder = 'Noise\\N'+str(N)
else:
    folder = 'No Noise\\N'+str(N)

#Create directory for files
base_path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\VQE\Adam\Files\{}\\{}\\".format(potential, folder)
os.makedirs(base_path, exist_ok=True)

print(f"Running for {potential} potential")

for cut_off in cut_offs_list:

    print(f"Running for cutoff: {cut_off}")

    #calculate Hamiltonian and expected eigenvalues
    H = calculate_wz_hamiltonian(cut_off, N, a, potential, boundary_condition, c)
    
    eigenvalues = np.sort(np.linalg.eig(H)[0][:12])
    min_eigenvalue = min(eigenvalues.real)

    #create qiskit Hamiltonian Pauli string
    hamiltonian = SparsePauliOp.from_operator(H)
    num_qubits = hamiltonian.num_qubits
    print("Num qubits: ", num_qubits)
    
    # Device
    shots = shots
    dev = qml.device('lightning.qubit', wires=num_qubits, shots=shots)


    #Initial params shape
    num_layers = 1
    params_shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)


    # Define the cost function
    @qml.qnode(dev)
    def cost_function(params):
        params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
        qml.StronglyEntanglingLayers(weights=params, wires=range(num_qubits), imprimitive=qml.CZ)
        return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))
    
        
    #Optimizer
    stepsize = 0.5
    optimizer = AdamOptimizer(stepsize=stepsize)

    # VQE
    vqe_start = datetime.now()

    #variables
    num_vqe_runs = 1
    max_iterations = 10000
    tolerance = 1e-8
    moving_avg_length = 5

    #data arrays
    energies = []
    param_values = []
    success = []
    run_times = []
    num_iters = []

    for j in range(num_vqe_runs):

        if j % 1==0:
            print(f"VQE run: {j}")

        run_start = datetime.now()
        converged = False
        prev_energy = None

        moving_average_check = False
        gradient_norm_check = False

        #Initial params
        scale = 0.25
        params_shape = qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=num_qubits)
        params = scale*np.pi * pnp.random.random(size=params_shape)

        iter_energies = []

        for i in range(max_iterations):

            params, energy = optimizer.step_and_cost(cost_function, params)
            iter_energies.append(energy)

            # Moving average convergence check
            if len(iter_energies) > moving_avg_length:
                energy_moving_avg = np.mean(np.abs(np.diff(iter_energies[-moving_avg_length:])))
                if energy_moving_avg < tolerance:
                    
                    moving_average_check = True
                    converged = True
                    break


            prev_energy = energy
        
        if converged == False:
            print("Not converged")

        energies.append(energy)
        param_values.append(params)
        success.append(converged)
        num_iters.append(i+1)

        run_end = datetime.now()
        run_time = run_end - run_start
        run_times.append(run_time)

    vqe_end = datetime.now()
    vqe_time = vqe_end - vqe_start


Running for linear potential
Running for cutoff: 4
Num qubits:  6
VQE run: 0
Not converged


In [15]:
energies

[array(0.18034391)]

In [16]:
eigenvalues

array([3.22505916e-05+0.j, 9.75406022e-01+0.j, 9.75406022e-01+0.j,
       1.11806624e+00+0.j, 1.11806624e+00+0.j, 2.23610023e+00+0.j,
       4.51389977e+00+0.j, 5.63193376e+00+0.j, 5.63193376e+00+0.j,
       5.77459398e+00+0.j, 5.77459398e+00+0.j, 6.74996775e+00+0.j])

In [4]:
#Save run
run = {
    'starttime': starttime,
    'potential': potential,
    'boundary_condition': boundary_condition,
    'c': c,
    'cutoff': cut_off,
    'num_sites': N,
    'exact_eigenvalues': [x.real.tolist() for x in eigenvalues],
    'ansatz': 'StronglyEntanglingLayers-1layer',
    'num_VQE': num_vqe_runs,
    'shots': shots,
    'Optimizer': {'name': 'AdamOptimizer',
                'stepsize':stepsize,
                'maxiter':max_iterations,
                'tolerance': tolerance,
                'moving_avg_length': moving_avg_length
                },
    'results': [x.tolist() for x in energies],
    'params': [x.tolist() for x in param_values],
    'num_iters': num_iters,
    'success': np.array(success, dtype=bool).tolist(),
    'run_times': [str(x) for x in run_times],
    'total_run_time': str(vqe_time)
}

# Save the variable to a JSON file
path = base_path + "{}_{}.json".format(potential, cut_off)
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)

In [ ]:
# create plots
base_path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\SUSY\PennyLane\SUSY VQE\Shot Noise\Adam\Files\{}\\{}\\"
create_vqe_plots(potential=potential, base_path=base_path, folder=folder, cut_off_list=cut_offs_list)